# Spark

In [1]:
import random
from pyspark import SparkContext
from pyspark.sql import SQLContext

import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import Window

import json
import numpy as np
import pandas as pd
from datetime import datetime
import re

sc = SparkContext(appName="NU_hackathon")
spark = SQLContext(sc)

True

# Lib

In [2]:
# PySpark
from pyspark.sql import functions as F, Window
from pyspark.sql.types import TimestampType, DoubleType, IntegerType, StringType, ArrayType, StructField, BooleanType, LongType, StructType, DataType
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
# широкий jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# pandas
import pandas as pd
# numpy
import numpy as np

# Functions

In [3]:
# преобразование нужных 2Гис категорий 
def gis_preprocessing_org_cat(gis_dm):
    # убираем неправильные координаты
    cond_null = (F.col("lat").isNotNull())&(F.col("lon").isNotNull())
    cond_max_lat_long = (F.col("lat") <= 90)&(F.col("lon") <= 180)
    gis_dm = gis_dm.filter(cond_null & cond_max_lat_long)
    
    
    # 1) категория - конкурирующие телеком операторы
    cond_kcell = (F.col("company_name").like("%kcell%"))
    cond_tele2 = ((F.col("company_name").like("%tele2%"))|(F.col("company_name").like("%altel%")))
    subcat_org_1 = gis_dm.\
    filter(cond_kcell | cond_tele2).\
    withColumn("ORG_SUBCAT", F.when(cond_kcell, "KCELL").otherwise("TELE2")).select("ORG_ID", "ORG_SUBCAT")

    # 2) целевые подкатегории
    subcat_org_2 = gis_dm.filter(~(cond_kcell | cond_tele2))
    # выдлеляем свои подкатегории для организаций с помощью ключевых слов
    # словарь - подкатегория: ключевые слова
    target_org_subcat_dict = {
        "Cafe": ["кафе", "кафе-кондитерские / кофейни"], # кафе
        "Restaurant": ["рестораны"], # рестораны
        "Bar": ["бары"], # бары
        "Beauty": ["парикмахерские", "услуги косметолога", "косметика / парфюмерия",
                    "косметика / расходные материалы для салонов красоты",
                   "услуги по уходу за ресницами / бровями", "услуги визажиста",
                   "услуги массажиста", "spa-процедуры",
                   "барбершопы"], # красота
        "TRC": ["торговые центры / универсальные магазины"], # ТРЦ, рынки
        "Microfinance": ["микрофинансирование"], # микрофинансы
        "Bank": ["банки"], # банки
        "ATM": ["банкоматы"], # банкоматы
        "Hotels": ["гостиницы"], # гостиницы
        "Hostels": ["хостелы"], # хостелы
        "Clothes_Shoes": ["женская одежда", "мужская одежда", "детская одежда", "спортивная одежда / обувь", "верхняя одежда", 
                   "спецодежда / средства индивидуальной защиты", "джинсовая одежда",
                   "одежда / обувь для танцев", "одежда / обувь для силовых структур",
                        "обувные магазины"], # одежда и обувь
        "Supermarkets": ["супермаркеты"], # супермаркеты
        "Products": ["молочные продукты", "продукты быстрого приготовления", "рыба / морепродукты, кондитерские изделия",
                    "алкогольные напитки", "точки безалкогольных напитков",
                    "безалкогольные напитки", "хлебобулочные изделия", "овощи / фрукты",
                    "мука / крупы", "мясо / полуфабрикаты", "мясо птицы / полуфабрикаты",
                    "колбасные изделия", "мороженое", "консервированная продукция",
                    "яйцо", "пекарни", "продовольственные магазины"], # продукты
        "Sport": ["спортивные секции", "спортивный инвентарь",
                 "спортивно-интеллектуальные клубы","спортивное оборудование",
                  "спортивное питание","аренда спортивных площадок", "спортивные школы",
                   "федерации спорта", "профессиональные спортивные клубы", "спортивно-тактические клубы"], # спорт
        "Education": ["гимназии", "начальные школы-детские сады / прогимназии", "лицеи", "языковые школы",
                     "школы", "детские музыкальные школы",
                     "детские художественные школы", "школы искусств", "киношколы", "школы-интернаты",
                      "частные детские сады", "детские сады", "колледжи", "университеты",
                      "научно-исследовательские институты", "институты", "студенческие общежития"], # образование
        "Government": ["государственные службы", "акиматы районов города",
                        "акимат города", "акимат области", "маслихат города"], # государство
        "Business_Center": ["бизнес-центры"] # бизнес центры
    }
    # DataFrame
    data = np.array([[k, v] for k, v in target_org_subcat_dict.items()]).reshape((len(target_org_subcat_dict), 2))
    org_subcat_df = pd.DataFrame(data = data, columns = ["ORG_SUBCAT", "TARGET_SUBCAT_LST"])
    org_subcat_df = spark.createDataFrame(org_subcat_df) 
    # subcat * ORG_SUBCAT
    subcat_org_2 = subcat_org_2.crossJoin(org_subcat_df)
    # отображение subcat -> ORG_SUBCAT
    @F.udf("int")
    def org_subcat_mapping(subcat_lst, target_subcact_lst):
        subcat_lst = [s.lower().strip() for s in subcat_lst]
        s1 = set(subcat_lst)
        s2 = set(target_subcact_lst)
        return len(s1.intersection(s2))
    
    subcat_org_2 = subcat_org_2.withColumn("INTERSECT_LEN", org_subcat_mapping("subcategory_list", "TARGET_SUBCAT_LST")).\
    filter(F.col("INTERSECT_LEN") > 0).select("ORG_ID", "INTERSECT_LEN", "ORG_SUBCAT").distinct()
    wnd = Window().partitionBy("ORG_ID").orderBy(F.desc("INTERSECT_LEN"))
    subcat_org_2 = subcat_org_2.withColumn("RANK", F.row_number().over(wnd)).filter("RANK = 1").\
    select("ORG_ID", "ORG_SUBCAT")
    # union
    subcat_org = subcat_org_1.union(subcat_org_2)
    
    # join
    res = gis_dm.join(subcat_org, on = ["ORG_ID"], how = "left").fillna(value = "N_A", subset = ["ORG_SUBCAT"])
    
    res = res.select('ORG_ID',
     'company_name',
     'description',
     F.col('subcategory_list').cast(T.StringType()),
     'lon',
     'lat',
     'city_name',
     'ORG_SUBCAT')
    
    return res


def ymaps_preprocessing_org_cat(ymaps):
    ymaps = ymaps.withColumn('mycol',F.col('businesses')[0].categories[0])

    cats = ymaps.select('mycol').distinct().toPandas()
    cats_splitted = list()
    for i in cats.mycol.values:
        if isinstance(i,type(None)):
            cats_splitted.append({'class':'N/A','id':'N/A','name':'N/A','pluralName':'N/A',	'seoname':'N/A'})   
        else:
            cats_splitted.append(i.asDict())
    cats_splitted = pd.DataFrame(cats_splitted)
    cats_splitted['mycol'] =  cats['mycol']

    giscats = {'Business_Center': ['Бизнес-центр', 'Коворкинг', 'Бизнес-инкубатор'],
    'Education': [
           'Дополнительное образование', 'Учебный комбинат', 'ВУЗ',
           'Хореографическое училище', 'Центр повышения квалификации',
           'Школа искусств', 'Общеобразовательная школа',
           'Театральное и цирковое образование',
           'Бизнес-школа', 'Лицей', 'Тренинги', 'Гимназия',
           'Школа санаторного типа', 'Начальная школа', 'Фотошкола', 'Техникум', 'Училище',
           'Обучение за рубежом', 'Частная школа', 'Воскресная школа',
           'Военная, кадетская школа', 'Учебный центр', 'Школа танцев',
           'Школа-интернат',
           'Духовное учебное заведение',
           'Музыкальное образование', 'Колледж'],
    'Bank': ['Банк'],
    'Hotels': ['Бронирование гостиниц','Гостиница','Жильё посуточно'],
    'Hostels': ['Хостел'],

    'Clothes_Shoes':['Спецодежда', 'Карнавальные, театральные и танцевальные костюмы',
           'Свадебный салон', 'Секонд-хенд', 'Одежда больших размеров',
           'Вещевой рынок', 'Магазин верхней одежды',
           'Магазин головных уборов', 'Войлочные и фетровые изделия',
           'Магазин обуви', 'Салон вечерней одежды',
           'Ателье по пошиву одежды', 'Магазин джинсовой одежды',
           'Магазин чулок и колготок', 'Меховое ателье', 'Военная форма',
           'Магазин кожи и меха', 'Магазин бижутерии', 'Магазин одежды'],
    'Government' : ['Гражданская оборона', 'Совет депутатов',
           'Органы государственного надзора',
           'МФЦ', 'Торгово-промышленная палата',
           'Министерства, ведомства, государственные службы',
           'Посольство, консульство', 'Политическая партия'],
    'Microfinance': ['Микрофинансирование'],
    'Supermarkets': ['Супермаркет'],
    'Products': ['Магазин мяса, колбас', 'Рынок',
           'Магазин суши и азиатских продуктов',
           'Мёд и продукты пчеловодства', 'Продуктовый рынок',
           'Магазин рыбы и морепродуктов', 'Магазин кулинарии',
           'Магазин сыров', 'Супермаркет', 'Магазин овощей и фруктов',
           'Диетические и диабетические продукты', 'Магазин чая и кофе',
           'Молочный магазин', 'Орехи, снеки, сухофрукты',
           'Магазин продуктов', 'Яйцо и мясо птицы', 'Мороженое',
           'Овощи и фрукты оптом'],
    'Restaurant':['Ресторан'],
    'Bar': ['Спортбар', 'Бар, паб', 'Ночной клуб', 'Кальян-бар'],
    'Beauty':['Салон бровей и ресниц', 'Визажисты, стилисты', 'Шугаринг',
           'Лазерная эпиляция',
           'Ногтевая студия', 'Сауна', 'Косметология',
           'Салон красоты', 'Солярий',
           'Спа-салон', 'Пластическая хирургия'],
    'TRC': ['Торговый центр'],
    'Sport': [ 'Аэроклуб',
           'Спортивная школа', 'Лыжная база', 'Киберспорт',
           'Спортивно-развлекательный центр', 'Лазертаг', 'Картинг',
           'Теннисный клуб', 'Спортивное объединение', 'Горнолыжный комплекс', 'Пейнтбол',
           'Страйкбол', 'Стрелковый клуб, тир'],
    'Cafe': ['Кафе', 'Антикафе', 'Кофейня', 'Бар безалкогольных напитков'],
    "ATM" : ['Банкомат']}


    giscats_for_df = []
    for i in giscats.keys():
        for j in giscats[i]:
            giscats_for_df.append({'ORG_SUBCAT':i,'name':j})

    giscats_df = pd.DataFrame(giscats_for_df)

    for_map = cats_splitted.merge(giscats_df, on='name', how='left')
    for_map['ORG_SUBCAT'] = for_map['ORG_SUBCAT'].fillna('N_A')
    for_map = spark.createDataFrame(for_map)
    ymaps = ymaps.join(for_map, on='mycol')
    ymaps = ymaps.withColumn('lon',F.col('coordinates')[0])
    ymaps = ymaps.withColumn('lat',F.col('coordinates')[1])

    ymaps_dm = ymaps.select(F.col('businesses')['title'].alias('company_name').cast(T.StringType()),
                        F.col('businesses').alias('description').cast(T.StringType()),
                        'lat', 'lon',
                        F.col('mycol').alias('subcategory_list').cast(T.StringType()),
                        F.col('compositeAddress')['locality'].alias('city_name'),
                        'ORG_SUBCAT')
    ymaps_dm = ymaps_dm.withColumn("ORG_ID", F.monotonically_increasing_id())

    cities_yandex = ['Шымкент','Тараз','Кызылорда','Семей','Талдыкорган','Туркестан']
    cond_null = (F.col("lat").isNotNull())&(F.col("lon").isNotNull())
    cond_max_lat_long = (F.col("lat") <= 90)&(F.col("lon") <= 180)
    ymaps_dm = ymaps_dm.filter(cond_null & cond_max_lat_long)
    ymaps_dm = ymaps_dm.filter(F.col('city_name').isin(cities_yandex))
    ymaps_dm = ymaps_dm.select('ORG_ID','company_name','description','subcategory_list','lon','lat','city_name','ORG_SUBCAT')
    
    return ymaps_dm

def gis_preprocessing_to_zid(grid, gis_dm):
    gis_dm.createOrReplaceTempView("gis_dm")
    gis_dm = spark.sql(
    '''
    SELECT *, (CAST(lon AS Decimal(24, 20)) + CAST(lat AS Decimal(24, 20))) as GEOM_POINT
    FROM  gis_dm
    '''
    )
    
    # 1.2 Распределение организаций по zid'ам
    gis_dm.createOrReplaceTempView("gis_dm")
    grid.createOrReplaceTempView("grid")
    
    gis_to_zid = spark.sql(
    '''
    SELECT tab1.*, tab2.ZID_ID
    FROM  gis_dm AS tab1, grid AS tab2
    WHERE 
    tab2.GEOM_POLYGON == tab1.GEOM_POINT
    '''
    )
    
    gis_to_zid = gis_to_zid.drop("GEOM_POINT")
    
    return gis_to_zid

# Input

In [19]:
# города
from email import header


cities_gis = ["Алматы", "Нур-Султан",
              "Караганда", "Павлодар",
              "Усть-Каменогорск","Актобе",
              "Уральск", "Костанай", "Атырау",
              "Актау", "Кокшетау", "Петропавловск",
              "Темиртау", "Аксу"]

cities_yandex = ['Шымкент','Тараз','Кызылорда','Семей','Талдыкорган','Туркестан']

schema = (StructType([StructField('name',StringType(),True),
            StructField('description',StringType(),True),
            StructField('city_name',StringType(),True),
            StructField('address',StringType(),True),
            StructField('website',StringType(),True),
            StructField('instagram',StringType(),True),
            StructField('VK',StringType(),True),
            StructField('subcategory_list', ArrayType(StringType(),True),True),
            StructField('id_2',LongType(),True),
            StructField('lon',StringType(),True),
            StructField('lat',StringType(),True),
            StructField('schedule',ArrayType(StringType(),True),True),
            StructField('phone_clean_arr',ArrayType(StringType(),True),True),
            StructField('stop_factors',ArrayType(StringType(),True),True)]))

gis_dm = spark.read.csv('../data/2gis_test.csv', header=True)
gis_dm = (
    gis_dm
    .filter((F.col("city_name").isin(cities_gis)))
    .select((F.lower(F.col("name"))).alias("company_name"),
            "description",
            "subcategory_list", F.col("id_2").alias("ORG_ID"),
            "lon", "lat", "city_name"))
grid_path = spark.read.csv('../data/polygons_test.csv', header=True)
grid_gis = grid_path.filter((F.col("DIAG_SIZE") == 750)&(F.col("CITY").isin(cities_gis)))
grid_ymaps = grid_path.filter((F.col("DIAG_SIZE") == 750)&(F.col("CITY").isin(cities_yandex)))
grid_gis.createOrReplaceTempView("grid_gis")
grid_gis = spark.sql(
'''
SELECT *, POLYGON_WKT as GEOM_POLYGON
FROM  grid_gis
'''
)

# Processing

In [20]:
# преобразовние_1: работа с катрегориями организаций
gis_dm = gis_preprocessing_org_cat(gis_dm)
# преобразование_2: отображение в zid
gis_dm = gis_preprocessing_to_zid(grid_gis, gis_dm)

/hdd/bigdata/python_envs/mobad/mobad/lib/python3.7/site-packages/ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [21]:
gis_dm.explain('formatted')

== Parsed Logical Plan ==
Project [ORG_ID#263L, company_name#262, description#234, subcategory_list#418, lon#242, lat#243, city_name#235, ORG_SUBCAT#409, ZID_ID#271]
+- AnalysisBarrier
      +- Project [ORG_ID#263L, company_name#262, description#234, subcategory_list#418, lon#242, lat#243, city_name#235, ORG_SUBCAT#409, GEOM_POINT#427, ZID_ID#271]
         +- Filter  **org.apache.spark.sql.geosparksql.expressions.ST_Contains$**
            +- Join Inner
               :- SubqueryAlias tab1
               :  +- SubqueryAlias gis_dm
               :     +- Project [ORG_ID#263L, company_name#262, description#234, subcategory_list#418, lon#242, lat#243, city_name#235, ORG_SUBCAT#409, st_point(cast(lon#242 as decimal(24,20)), cast(lat#243 as decimal(24,20))) AS GEOM_POINT#427]
               :        +- SubqueryAlias gis_dm
               :           +- Project [ORG_ID#263L, company_name#262, description#234, cast(subcategory_list#240 as string) AS subcategory_list#418, lon#242, lat#243, ci